In [2]:
!pip install scikit-posthocs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import sys
import numpy
numpy.set_printoptions(threshold=5)

In [5]:
algo1 = "/content/drive/Shareddrives/ML./project/output table/results_table_algo1.csv"
algo2 = "/content/drive/Shareddrives/ML./project/output table/results_table_algo2.csv"
algo2_new = "/content/drive/Shareddrives/ML./project/output table/results_table_new_algo2.csv"
mrmr = "/content/drive/Shareddrives/ML./project/output table/results_table_mRMR.csv"
selectFDR = "/content/drive/Shareddrives/ML./project/output table/results_table_selectFdr.csv"
relifeF = "/content/drive/Shareddrives/ML./project/output table/results_table_reliefF.csv"
rfe = "/content/drive/Shareddrives/ML./project/output table/results_table_RFE.csv"

In [6]:
algo1_df = pd.read_csv(algo1, index_col = 0)
algo2_df = pd.read_csv(algo2, index_col = 0)
algo2_new_df = pd.read_csv(algo2_new, index_col = 0)
mrmr_df = pd.read_csv(mrmr, index_col = 0)
selectFDR_df = pd.read_csv(selectFDR, index_col = 0)
relifeF_df = pd.read_csv(relifeF, index_col = 0)
rfe_df = pd.read_csv(rfe, index_col = 0)

In [7]:
algos = [algo1_df,algo2_df,algo2_new_df,mrmr_df,selectFDR_df,relifeF_df,rfe_df]
dss = algo1_df['Dataset Name'].unique()

In [8]:
l = []
for dataset in dss:  
  ds_res = pd.DataFrame()
  for algo in algos:
    name = algo['Dataset Name'] == dataset
    temp = algo[name]
    ds_res = ds_res.append(temp, ignore_index=True)
  l.append(ds_res)

In [9]:
k=[]
for tmp in l:
  tmp['Measure Value'] = tmp['Measure Value'].astype(float)
  res = tmp.groupby(['Dataset Name','Number of samples','Original Number of features','Filtering Algorithm','CV Method','Measure Type'], as_index=False)['Measure Value'].mean()
  k.append(res)

In [10]:
res = []
for p in k:
  ds2 = p['Dataset Name'] == 'DLBCL'
  acc = p['Measure Type'] == 'ACC'
  auc = p['Measure Type'] == 'AUC'
  loo = p['CV Method'] == 'split_LOO'
  p = p[(loo & acc) | (ds2 & acc) | auc]
  res.append(p.sort_values(['Measure Value'],axis=0, ascending=False))


In [11]:
new_res = []
for df in res:
  new_df = df.dropna()
  new_res.append(new_df)
all_df = pd.concat(new_res, ignore_index=True)

In [27]:
relevant_features = ["Dataset Name", "Filtering Algorithm", "Measure Value"]
filtered_df = all_df[relevant_features]

algo_names = filtered_df['Filtering Algorithm'].unique()

groups = []
for algo_name in algo_names:
  filtering_algo_df = filtered_df[filtered_df["Filtering Algorithm"] == algo_name]
  ds_names =  filtering_algo_df["Dataset Name"].array
  scores = filtering_algo_df["Measure Value"].array
  groups.append(scores)
groups = np.array(groups)
df = pd.DataFrame(groups)
df.columns = ds_names
df.index = algo_names
df.round(decimals=3).T


,selectFdr,mRMR,algo1,RFE,algo2,reliefF,new_algo2
CLL,0.773,0.770,0.747,0.742,0.562,0.499,0.398
COPDSexualDimorphism,0.807,0.830,0.725,0.762,0.740,0.587,0.622
DLBCL,0.399,0.365,0.360,0.392,0.388,0.392,0.386
Leukemia_3c_arff,0.923,0.901,0.630,0.913,0.809,0.568,0.557
Leukemia_4c_arff,0.864,0.810,0.453,0.830,0.805,0.566,0.559
Prostate,0.953,0.955,0.930,0.948,0.888,0.889,0.723
breastCancerVDX,0.929,0.929,0.893,0.935,0.912,0.578,0.771
breast_arff,0.646,0.662,0.719,0.688,0.663,0.539,0.565
colon,0.803,0.813,0.657,0.719,0.756,0.803,0.634
curatedOvarianData,0.445,0.456,0.525,0.505,0.524,0.463,0.506


In [13]:
from scipy import stats
import scikit_posthocs as sp

print(stats.friedmanchisquare(*groups))

FriedmanchisquareResult(statistic=49.689008042895495, pvalue=5.42656640138354e-09)


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [14]:
df = sp.posthoc_nemenyi_friedman(groups.T)
df.columns = algo_names
df.index = algo_names
df.round(decimals=4)


,selectFdr,mRMR,algo1,RFE,algo2,reliefF,new_algo2
selectFdr,1.0000,0.9000,0.0092,0.9000,0.8033,0.0015,0.0010
mRMR,0.9000,1.0000,0.0104,0.9000,0.8248,0.0017,0.0010
algo1,0.0092,0.0104,1.0000,0.0134,0.3393,0.9000,0.9000
RFE,0.9000,0.9000,0.0134,1.0000,0.8678,0.0023,0.0010
algo2,0.8033,0.8248,0.3393,0.8678,1.0000,0.1259,0.0172
reliefF,0.0015,0.0017,0.9000,0.0023,0.1259,1.0000,0.9000
new_algo2,0.0010,0.0010,0.9000,0.0010,0.0172,0.9000,1.0000
